<h1 align="center">
    <b style="color: green;">Trino </b><b style="color: red;">Iceberg </b><b style="color: black;">Example</b>
</h1>

---

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.schema import Table, MetaData
from sqlalchemy.sql.expression import select, text
import pandas as pd

In [ ]:
engine = create_engine('trino://root:@192.168.10.210:8090/iceberg/iceberg')
connection = engine.connect()

## Re-Snapshot

In [ ]:
connection.execute("CALL iceberg.system.rollback_to_snapshot('iceberg_test', 'tbl_example', 822704853044330400)")

## Query เพื่อดูประวัติการ Snapshot

In [ ]:
sql = '''
    SELECT
        *
    FROM
        iceberg.iceberg_test."tbl_example$snapshots"
    ORDER BY
        committed_at DESC
'''
pd.read_sql_query(sql, con=engine)

## แสดงรายการ Record หรือ Rows ทั้งหมดในตารางพร้อมระบุเวลาที่เกิด Condition พร้อมบอกว่า ข้อมูล Row นี้อยู่ในไฟล์ parquet ตัวไหน

In [ ]:
sql = '''
    SELECT
        *, 
        "$path", "$file_modified_time"
    FROM
        iceberg.iceberg_test.tbl_example
'''
pd.read_sql_query(sql, con=engine)

## Query แบบระบุไฟล์ Parquet

In [ ]:
sql = '''
    SELECT
        *
    FROM
        iceberg.iceberg_test.tbl_example
    WHERE
        "$path" = 'hdfs://nn01.bigdata:8020/user/hive/warehouse/iceberg_test/tbl_example/data/20230318_073101_00095_hdwqq-ae0b490d-8fd9-46ab-aed5-526bf34f0ea9.parquet'
'''
pd.read_sql_query(sql, con=engine)